In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your Lang API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs necessary to perform the action.

After executing actions, the results can be fed back into the LLM to determine whether more actions are needed, or whether it is okay to finish.

In [27]:
# Import relevant functionality
from langchain_community.tools.tavily_search import TavilySearchResults

# Create the agent
search = TavilySearchResults(max_results=2)

# print(search.invoke("What's a monad"))

# list of tools to use later
tools = [search]

In [28]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [29]:
# we give the model knowledge of the tools
model_with_tools = model.bind_tools(tools)

from langchain_core.messages import HumanMessage

# won't ue the tools here, there is no need
response = model_with_tools.invoke([HumanMessage(content="Hi!")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

# will use the tools here (more precisely call! It wants us to call the Tavily Search tool.
# This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Carrara (MS)?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []
ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Carrara MS'}, 'id': 'call_eZzq8lACLQ2zGURbHUKiBfWh', 'type': 'tool_call'}]


In [30]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

# bind_tools will be called under the hood
# without checkpointer it will have no memory
agent_executor = create_react_agent(model, tools, checkpointer=memory)

response = agent_executor.invoke(
  {"messages": [HumanMessage(content="Hi there")]},
  config={"configurable": {"thread_id": "abc123"}}
)
print(response["messages"])

config = {"configurable": {"thread_id": "abc123"}}

# stream -> we'll see all the intermediate steps
# same config -> same memory
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in Pisa, Italy")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live? what about the temperature?")]}, config
):
    print(chunk)
    print("----")

[HumanMessage(content='Hi there', additional_kwargs={}, response_metadata={}, id='8b22dc8d-8aad-44fe-8bdf-b0a0aa276a14'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d22ea7c7-7165-4a81-a1a2-fd0a5474c2a6-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]
{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you. Pisa, Italy is a beautiful city known for its iconic lean